In [5]:
import numpy as np
#import scipy.stats
#import seaborn as sns
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
#import scikit_posthocs as sp
#import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from matplotlib.transforms import BlendedGenericTransform
#import scikit_posthocs as sp
#from urllib.request import urlopen
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import os

import keras
import keras.layers as layers
import numpy as np

TODO
Create data from clips
Create iterator to load data for all clips

Gener

In [30]:
# some test data to hack around with
wav, rate = librosa.core.load("Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav")
fft = librosa.stft(wav)

In [59]:
# Creating data from clip wave file

WINDOW_SIZE = 10

def clip_frames(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    frames = []
    for i in range(half_win, fft.shape[0] - half_win):
        frames.append(fft[i:i+WINDOW_SIZE])
    return frames

def clip_targets(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    targets = []
    for i in range(half_win, fft.shape[0] - half_win):
        targets.append(fft[i+half_win:i+half_win+1])
    return targets

In [67]:
def create_data(root):
    clean_dir = root + "\\Clean\\"
    noisy_dir = root + "\\Noisy\\"
    frames = []
    targets = []
    for file in os.listdir(clean_dir)[0:10]:
        filename = os.fsdecode(file)
        frames = frames + clip_frames(noisy_dir + file)
        targets = targets + clip_targets(clean_dir + file)
    return frames, targets


In [69]:
#clip_windows("Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav")
frames, targets = create_data("Assets\\DataShareArchive\\Test")

In [70]:
print(frames[0][5][0])

print(targets[0][0][0])

(-0.04013502-5.551115e-17j)
(0.16505659+1.3877788e-17j)


In [34]:
fft[0:1].shape

(1, 120)

In [2]:
model = keras.Sequential(
     [layers.Dense(64, activation='relu', input_shape=[2]),
      layers.Dense(16, activation='relu'),
     layers.Dense(1, activation='linear')]
)

In [66]:
os.listdir("Assets\\DataShareArchive\\Test\\Noisy\\")[0:5]

['p232_001.wav',
 'p232_002.wav',
 'p232_003.wav',
 'p232_005.wav',
 'p232_006.wav']